In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# so that we can import `tools.py`
# Navigate two directories up from the current working directory
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# Add this directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pickle
from tqdm import tqdm

In [3]:
# loading a sample from our TinyStories dataset
from tools import get_data_loader
data_loader = get_data_loader(batch_size=10_000, split='train')

batch = next(iter(data_loader))
print(len(batch), batch[0])

Found cached dataset json (/Users/tunadorable/.cache/huggingface/datasets/noanabeshima___json/noanabeshima--TinyStoriesV2-226173b7dd235c68/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


10000 Tom and Anna like to play with books. They have many books in their room. Some books have pictures, some books have stories, some books have songs. Tom and Anna like to look at the books and learn new words.
One day, Tom and Anna find a big book on the shelf. It is very heavy and has many pages. It is a dictionary. Tom and Anna do not know what a dictionary is. They open the book and see many words. Some words they know, some words they do not know. They are unknown words.
"Look, Anna, this word is 'lie'. Do you know what it means?" Tom asks.
Anna shakes her head. She does not know what 'lie' means.
"Maybe it means to tell something that is not true. Like when you say you ate your broccoli, but you did not," Tom guesses.
Anna frowns. She does not like when Tom says that. She did eat her broccoli, but only a little bit.
"No, Tom, that is not what 'lie' means. Maybe it means to lie down on the bed. Like when you are sleepy or sick," Anna says.
Tom nods. He thinks that makes sense. 

In [4]:
# turn it into one string instead of a list of strings
combined_string = '\n\n'.join(batch)

# find the unique characters
chars = sorted(list(set(combined_string)))
# this is the largest set of characters i found earlier from a batch size of 1_000_000, total 95 characters
if len(chars) < 95:
    chars = ['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
v = len(chars)
print('\n', chars, v)


 ['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~'] 95


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
char_encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers

# Regex
this is a pre-processing stage where we set the rules for what types of characters are allowed to be merged together

In [6]:
import regex as re

In [7]:
# don't ask me the specifics of how this plays out, i just know it's what they used for GPT4
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
# if you want to mess around with building your own tokenizer, then ^this string is one of the things to mess around with

In [8]:
compiled_pattern = re.compile(GPT4_SPLIT_PATTERN)
print(compiled_pattern)

regex.Regex("'(?i:[sdmt]|ll|ve|re)|[^\\r\\n\\p{L}\\p{N}]?+\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]++[\\r\\n]*|\\s*[\\r\\n]|\\s+(?!\\S)|\\s+", flags=regex.V0)


In [9]:
# split the text up into text chunks
text_chunks = re.findall(compiled_pattern, combined_string)
print(len(combined_string), len(text_chunks))

8105281 1921543


In [10]:
print(combined_string[:100])
print(text_chunks[:100])

Tom and Anna like to play with books. They have many books in their room. Some books have pictures, 
['Tom', ' and', ' Anna', ' like', ' to', ' play', ' with', ' books', '.', ' They', ' have', ' many', ' books', ' in', ' their', ' room', '.', ' Some', ' books', ' have', ' pictures', ',', ' some', ' books', ' have', ' stories', ',', ' some', ' books', ' have', ' songs', '.', ' Tom', ' and', ' Anna', ' like', ' to', ' look', ' at', ' the', ' books', ' and', ' learn', ' new', ' words', '.\n', 'One', ' day', ',', ' Tom', ' and', ' Anna', ' find', ' a', ' big', ' book', ' on', ' the', ' shelf', '.', ' It', ' is', ' very', ' heavy', ' and', ' has', ' many', ' pages', '.', ' It', ' is', ' a', ' dictionary', '.', ' Tom', ' and', ' Anna', ' do', ' not', ' know', ' what', ' a', ' dictionary', ' is', '.', ' They', ' open', ' the', ' book', ' and', ' see', ' many', ' words', '.', ' Some', ' words', ' they', ' know', ',', ' some']


In [11]:
# input text preprocessing
ids = [char_encode(ch) for ch in text_chunks] # list(ch.encode("utf-8")) # <- use that instead to do actual bytez instead of characters
ids_backup = ids # saving this for later just to see how much compression we get
print(len(ids), ids[:100])

1921543 [[53, 79, 77], [2, 65, 78, 68], [2, 34, 78, 78, 65], [2, 76, 73, 75, 69], [2, 84, 79], [2, 80, 76, 65, 89], [2, 87, 73, 84, 72], [2, 66, 79, 79, 75, 83], [16], [2, 53, 72, 69, 89], [2, 72, 65, 86, 69], [2, 77, 65, 78, 89], [2, 66, 79, 79, 75, 83], [2, 73, 78], [2, 84, 72, 69, 73, 82], [2, 82, 79, 79, 77], [16], [2, 52, 79, 77, 69], [2, 66, 79, 79, 75, 83], [2, 72, 65, 86, 69], [2, 80, 73, 67, 84, 85, 82, 69, 83], [14], [2, 83, 79, 77, 69], [2, 66, 79, 79, 75, 83], [2, 72, 65, 86, 69], [2, 83, 84, 79, 82, 73, 69, 83], [14], [2, 83, 79, 77, 69], [2, 66, 79, 79, 75, 83], [2, 72, 65, 86, 69], [2, 83, 79, 78, 71, 83], [16], [2, 53, 79, 77], [2, 65, 78, 68], [2, 34, 78, 78, 65], [2, 76, 73, 75, 69], [2, 84, 79], [2, 76, 79, 79, 75], [2, 65, 84], [2, 84, 72, 69], [2, 66, 79, 79, 75, 83], [2, 65, 78, 68], [2, 76, 69, 65, 82, 78], [2, 78, 69, 87], [2, 87, 79, 82, 68, 83], [16, 1], [48, 78, 69], [2, 68, 65, 89], [14], [2, 53, 79, 77], [2, 65, 78, 68], [2, 34, 78, 78, 65], [2, 70, 73, 78,

so this regex just splits the text up into all the token ids that are allowed to be merged, meaning that the regex output we saw above is an upper limit on the tokens that we could end up with if we get a large enough vocabulary, rather than a starting point. ngl the reason I didn't use regex on bpe_v1 is because I thought it was a starting point and didn't understand how subwords were supposed to work -_-

# CPE tokenization
CPE = character-pair encoding instead of byte-pair encoding. Honestly we could've done BPE and the only difference would've been a negligibly larger embedding tensor in our model (specifically 256-95=161 extra rows) but might as well since we know we're only gonne be using this one same dataset

In [12]:
vocab_size = 8192 # the desired final vocabulary size
num_merges = vocab_size - v

In [13]:
# most models work off bytes, but we'll be simplifying to just the index of each unique character
base_indices = char_encode(chars)
print(base_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]


In [14]:
def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [15]:
# now let's actually do it
merges = {} # (int, int) -> int
for i in tqdm(range(num_merges)):
    # count the number of times every consecutive pair appears
    stats = {}
    for chunk_ids in ids:
        # passing in stats will update it in place, adding up counts
        get_stats(chunk_ids, stats)
    # find the pair with the highest count
    pair = max(stats, key=stats.get)
    # mint a new token: assign it the next available id
    idx = v + i
    # replace all occurrences of pair in ids with idx
    ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
    # save the merge
    merges[pair] = idx
    #print(f"merge {i+1}/{num_merges}: {pair} -> {idx} had {stats[pair]} occurrences")

100%|██████████████████| 8097/8097 [3:05:09<00:00,  1.37s/it]


In [16]:
og = sum([len(t) for t in ids_backup])
new = sum([len(t) for t in (ids)])
print("original length:", og) # remember tokens are our original tokens
print("ids length:", new) # and ids are new tokens we've made
print(f"compression ratio: {og / new:.2f}X")

original length: 8105281
ids length: 1929993
compression ratio: 4.20X


In [31]:
# Ensure the tokenizers directory exists
if not os.path.exists('./models'):
    os.makedirs('./models')

# Prepare the tokenizer data to be saved
tokenizer_data = {
    'stoi': stoi,  # Character to integer mapping
    'merges': merges  # Merges dictionary
}

# Save the tokenizer data using pickle
with open(f'./models/{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

In [30]:
# taking a pre-existing tokenizer and trimming it down to a smaller size
# i basically ran this cell and then one above it multiple times until i got to the smallest possible size (128)
vocab_size = vocab_size // 2 # len(chars) for character-wise tokenization
merges = {k: v for k, v in merges.items() if v < vocab_size}

In [32]:
from tokenizer import load_tokenizer_data, BPE_Tokenizer
vocab_size = 8192
tokenizer_data = load_tokenizer_data(f'models/{vocab_size}.model')
tokenizer = BPE_Tokenizer(tokenizer_data['stoi'], tokenizer_data['merges'])

In [33]:
prompt = 'Once upon a time there was a boy named Tim.'
print(tokenizer.display(prompt))

Once| upon| a| tim|e| th|er|e| was| a| boy| na|me|d| Tim|.


In [34]:
print(tokenizer.display(batch[0]))

To|m| an|d| An|na| like| to| play| with| bo|o|k|s|.| Th|ey| have| ma|ny| bo|o|k|s| in| th|e|ir| room|.| So|me| bo|o|k|s| have| pi|ct|ur|es|,| so|me| bo|o|k|s| have| sto|ries|,| so|me| bo|o|k|s| have| so|ng|s|.| To|m| an|d| An|na| like| to| lo|o|k| at| th|e| bo|o|k|s| an|d| lea|r|n| new| wo|r|d|s|.
|One| day|,| To|m| an|d| An|na| fi|nd| a| bi|g| bo|o|k| o|n| th|e| sh|el|f|.| It| is| v|ery| h|e|avy| an|d| has| ma|ny| pa|ge|s|.| It| is| a| di|ct|io|na|ry|.| To|m| an|d| An|na| do| no|t| know| what| a| di|ct|io|na|ry| is|.| Th|ey| open| th|e| bo|o|k| an|d| see| ma|ny| wo|r|d|s|.| So|me| wo|r|d|s| th|ey| know|,| so|me| wo|r|d|s| th|ey| do| no|t| know|.| Th|ey| ar|e| unknown| wo|r|d|s|.
|"L|ook|,| An|na|,| this| wo|r|d| is| '|lie|'.| Do| you| know| what| it| means|?"| To|m| asks|.
|An|na| shakes| h|er| h|ead|.| Sh|e| does| no|t| know| what| '|lie|'| means|.
|"M|aybe| it| means| to| te|ll| so|me|th|ing| that| is| no|t| tru|e|.| Like| wh|en| you| sa|y| you| ate| your| broccoli|,| bu|t| you| di|

In [35]:
for i in range(vocab_size):
    print(f"{i}: '{tokenizer.decode([i])}'")

0: '	'
1: '
'
2: ' '
3: '!'
4: '"'
5: '#'
6: '$'
7: '%'
8: '&'
9: '''
10: '('
11: ')'
12: '*'
13: '+'
14: ','
15: '-'
16: '.'
17: '/'
18: '0'
19: '1'
20: '2'
21: '3'
22: '4'
23: '5'
24: '6'
25: '7'
26: '8'
27: '9'
28: ':'
29: ';'
30: '<'
31: '='
32: '>'
33: '?'
34: 'A'
35: 'B'
36: 'C'
37: 'D'
38: 'E'
39: 'F'
40: 'G'
41: 'H'
42: 'I'
43: 'J'
44: 'K'
45: 'L'
46: 'M'
47: 'N'
48: 'O'
49: 'P'
50: 'Q'
51: 'R'
52: 'S'
53: 'T'
54: 'U'
55: 'V'
56: 'W'
57: 'X'
58: 'Y'
59: 'Z'
60: '['
61: '\'
62: ']'
63: '_'
64: '`'
65: 'a'
66: 'b'
67: 'c'
68: 'd'
69: 'e'
70: 'f'
71: 'g'
72: 'h'
73: 'i'
74: 'j'
75: 'k'
76: 'l'
77: 'm'
78: 'n'
79: 'o'
80: 'p'
81: 'q'
82: 'r'
83: 's'
84: 't'
85: 'u'
86: 'v'
87: 'w'
88: 'x'
89: 'y'
90: 'z'
91: '{'
92: '|'
93: '}'
94: '~'
95: ' t'
96: 'he'
97: ' a'
98: ' s'
99: ' w'
100: 'nd'
101: ' the'
102: 'ed'
103: ' b'
104: ' to'
105: ' and'
106: ' h'
107: ' f'
108: 'in'
109: ' T'
110: ' wa'
111: 're'
112: 'it'
113: 'ou'
114: ' l'
115: ' d'
116: ' c'
117: ' p'
118: 'ay'
119: ' m'